In [21]:
# General Stuff:
import numpy as np
import pandas as pd

# Our Stuff:
from models.AutoRecBase import AutoRecBase
from models.VarAutoRec import VarAutoRec
from models.MF import MF
from models.UserBiasAE import UserBiasAE


from scripts.get_data import download_2_data_sets, ratings_to_train_test, ratings_to_train_test_u
from scripts.get_2_other_data import get_2_other_datasets, secondary_to_train_test, secondary_to_train_test_u
from utils.evaluate import evaluate_model
from utils.loading_utils import load_model, save_model

import torch
from torch import nn
import pytorch_lightning as pl

# Visualization Stuff
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 240)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
if torch.cuda.is_available():
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1660 Ti


# AutoRec - AutoEncoders Meet Collaborative Filtering - PyTorch

<a id="toc"></a>
## Table of Content
1. [Introduction](#introduction)
1. [Conclusions](#conclusions)

<a id="introduction"></a>

## Introduction

In this notebook we will review a collaborative filtering approach using autoencoders, as suggested by Sedhain et al, in their 2015 paper "AutoRec: Autoencoders Meet Collaborative Filtering".

We will then introduce several improvements and asses them.

[Table of content](#toc)

In [22]:
is_default_dataset = False
if is_default_dataset:
    download_2_data_sets()
else:
    get_2_other_datasets()

In [23]:
if is_default_dataset:
    train_loader, val_loader = ratings_to_train_test(1,0, 1,10)
    mf_train_loader, mf_val_loader =  ratings_to_train_test_u(dataset_size=1,
                                                              validation_partition=0,
                                                              train_partition=1,
                                                              batch_size=10)
else:
    train_loader, val_loader = secondary_to_train_test('flixster',0, 1,10)
    mf_train_loader, mf_val_loader =  secondary_to_train_test_u('flixster',
                                                                validation_partition=0,
                                                                train_partition=1,
                                                                batch_size=10)


In [24]:
%reload_ext tensorboard
%tensorboard --logdir lightning_logs

Launching TensorBoard...

Go to:  [TensorBoard](http://localhost:6006)

In [40]:
models_dict = {} # (model,ephoc,lr): loss
models_state = {} # (model,ephoc,lr): model.state_dict()
models = [
    AutoRecBase,
    VarAutoRec,
    MF,
    UserBiasAE
]
lrs = [0.001,0.005,0.01]
activations = [nn.PReLU, nn.Sigmoid]

Sanity check:

In [27]:
for x, y, m in val_loader:
    print(x[:,0])
    break
for x, y, m in mf_val_loader:
    print(x[0])
    break
for x, y, r in mf_train_loader:
    print(x)
    print(y)
    print(r)
    break
# model(x)[:,0]

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])
tensor(7)
tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7])
tensor([ 929, 1000, 1135, 1165, 1183, 1187, 1291, 1561, 1576, 1577])
tensor([5.0000, 5.0000, 4.5000, 5.0000, 3.5000, 4.5000, 4.5000, 4.5000, 4.0000,
        4.0000], dtype=torch.float64)


In [28]:
print(f"Number of users: {len(val_loader.dataset)}")
print(f"Number of items: {len(val_loader.dataset[0][1])}")

Number of users: 402
Number of items: 2876


Datasets:

In [29]:
number_of_users = len(val_loader.dataset)
number_of_items = len(val_loader.dataset[0][1])

From the original paper:

In [30]:
latent_dims = [10, 80, 300]
lambdas = [0.01, 0.1, 1, 100]

In [41]:
should_train = True
model_paths = []
with torch.autograd.set_detect_anomaly(True):
    if should_train:
        for model_class in models:
            for lr in lrs:
                for latent in latent_dims:
                    for activation in activations:
                        for λ in lambdas:
                            model = model_class(number_of_items=number_of_items,
                                                num_of_users=number_of_users,
                                                hidden_size=latent,
                                                activation_function_1=activation,
                                                activation_function_2=activation,
                                                loss=nn.MSELoss(reduction='none'),
                                                λ=λ,
                                                lr=lr)
                            # training
                            trainer = pl.Trainer(gpus=0, max_epochs=10)
                            try:
                                if type(model).__name__ == "MF":
                                    trainer.fit(model,mf_train_loader, mf_val_loader)
                                else:
                                    trainer.fit(model,train_loader, val_loader)

                                model_path = save_model(model_class=model_class,
                                           trainer=trainer,
                                           activation=activation,
                                           hidden_size=latent,
                                           lr=lr,
                                           λ=λ,
                                           is_default_dataset=is_default_dataset)
                                model_paths.append(model_path)
                            except:
                                pass
        #                 break
        #             break
        #         break
        #     break
        # break
print(model_paths)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 28.8 K
1 | act_1     | PReLU   | 1     
2 | decoder   | Linear  | 31.6 K
3 | act_2     | PReLU   | 1     
4 | loss_func | MSELoss | 0     
--------------------------------------
60.4 K    Trainable params
0         Non-trainable params
60.4 K    Total params
0.242     Total estimated model params size (MB)


0.01
Epoch 0:  50%|█████     | 41/82 [00:00<00:00, 106.30it/s, loss=12, v_num=218]  
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████     | 41/82 [00:00<00:00, 104.59it/s, loss=10.1, v_num=218]
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 41/82 [00:00<00:00, 106.96it/s, loss=8.08, v_num=218]
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 41/82 [00:00<00:00, 107.33it/s, loss=6.13, v_num=218]
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 41/82 [00:00<00:00, 99.61it/s, loss=4.83, v_num=218] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  50%|█████     | 41/82 [00:00<00:00, 102.98it/s, loss=3.58, v_num=218]
Validating: 0it [00:00, ?it/s]
Epoch 6:  50%|█████     | 41/82 [00:00<00:00, 112.93it/s, loss=2.81, v_num=218]
Validating: 0it [00:00, ?it/s]
Epoch 7:  50%|█████     | 41/82 [00:00<00:00, 99.96it/s, loss=2.42, v_num=218] 
Validating: 0it [00:00, ?it/s]
Epoch 8:  50%|█████     | 41/82 [00:00<00:00, 97.82it/s, loss=2.06, v_num=218] 
Validating: 0it [00:00, ?it

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


0.1



  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 28.8 K
1 | act_1     | PReLU   | 1     
2 | decoder   | Linear  | 31.6 K
3 | act_2     | PReLU   | 1     
4 | loss_func | MSELoss | 0     
--------------------------------------
60.4 K    Trainable params
0         Non-trainable params
60.4 K    Total params
0.242     Total estimated model params size (MB)


...


0.1
Epoch 0:  50%|█████     | 41/82 [00:00<00:00, 41.18it/s, loss=1.08, v_num=503]
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████     | 41/82 [00:01<00:01, 39.02it/s, loss=1.09, v_num=503]
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 41/82 [00:00<00:00, 44.13it/s, loss=1.19, v_num=503]
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 41/82 [00:01<00:01, 39.65it/s, loss=1.12, v_num=503]
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 41/82 [00:01<00:01, 40.83it/s, loss=1.13, v_num=503]
Validating: 0it [00:00, ?it/s]
Epoch 5:  50%|█████     | 41/82 [00:00<00:00, 41.73it/s, loss=1.09, v_num=503]
Validating: 0it [00:00, ?it/s]
Epoch 6:  50%|█████     | 41/82 [00:01<00:01, 39.72it/s, loss=1.17, v_num=503]
Validating: 0it [00:00, ?it/s]
Epoch 7:  50%|█████     | 41/82 [00:00<00:00, 41.64it/s, loss=1.17, v_num=503]
Validating: 0it [00:00, ?it/s]
Epoch 8:  50%|█████     | 41/82 [00:01<00:01, 35.57it/s, loss=1.13, v_num=503]
Validating: 0it [00:00, ?it/s]
Epoch

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 863 K 
1 | act_1     | Sigmoid | 0     
2 | decoder   | Linear  | 865 K 
3 | act_2     | Sigmoid | 0     
4 | loss_func | MSELoss | 0     
--------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.915     Total estimated model params size (MB)



1
Epoch 0:  51%|█████     | 42/82 [00:01<00:01, 39.70it/s, loss=1.12, v_num=504]
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████     | 41/82 [00:00<00:00, 41.91it/s, loss=1.08, v_num=504]
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 41/82 [00:01<00:01, 36.09it/s, loss=1.08, v_num=504]
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 41/82 [00:01<00:01, 37.72it/s, loss=1.14, v_num=504]
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 41/82 [00:01<00:01, 37.62it/s, loss=1.17, v_num=504]
Validating: 0it [00:00, ?it/s]
Epoch 5:  50%|█████     | 41/82 [00:01<00:01, 37.14it/s, loss=1.15, v_num=504]
Validating: 0it [00:00, ?it/s]
Epoch 6:  50%|█████     | 41/82 [00:01<00:01, 36.25it/s, loss=1.14, v_num=504]
Validating: 0it [00:00, ?it/s]
Epoch 7:  50%|█████     | 41/82 [00:01<00:01, 36.44it/s, loss=1.1, v_num=504] 
Validating: 0it [00:00, ?it/s]
Epoch 8:  50%|█████     | 41/82 [00:01<00:01, 37.54it/s, loss=1.1, v_num=504] 
Validating: 0it [00:00, ?it/s]
Epoch 9

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 863 K 
1 | act_1     | Sigmoid | 0     
2 | decoder   | Linear  | 865 K 
3 | act_2     | Sigmoid | 0     
4 | loss_func | MSELoss | 0     
--------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.915     Total estimated model params size (MB)



100
Epoch 0:  50%|█████     | 41/82 [00:01<00:01, 38.57it/s, loss=1.09, v_num=505]
Validating: 0it [00:00, ?it/s]
Epoch 1:  51%|█████     | 42/82 [00:01<00:01, 39.10it/s, loss=1.1, v_num=505] 
Validating: 0it [00:00, ?it/s]
Epoch 2:  51%|█████     | 42/82 [00:01<00:01, 39.64it/s, loss=1.2, v_num=505]  
Validating: 0it [00:00, ?it/s]
Epoch 3:  51%|█████     | 42/82 [00:01<00:00, 41.25it/s, loss=1.09, v_num=505]
Validating: 0it [00:00, ?it/s]
Epoch 4:  51%|█████     | 42/82 [00:01<00:00, 40.90it/s, loss=1.12, v_num=505]
Validating: 0it [00:00, ?it/s]
Epoch 5:  51%|█████     | 42/82 [00:01<00:01, 38.32it/s, loss=1.06, v_num=505]
Validating: 0it [00:00, ?it/s]
Epoch 6:  50%|█████     | 41/82 [00:01<00:01, 37.99it/s, loss=1.1, v_num=505] 
Validating: 0it [00:00, ?it/s]
Epoch 7:  50%|█████     | 41/82 [00:01<00:01, 38.52it/s, loss=1.25, v_num=505]
Validating: 0it [00:00, ?it/s]
Epoch 8:  50%|█████     | 41/82 [00:01<00:01, 40.44it/s, loss=1.03, v_num=505]
Validating: 0it [00:00, ?it/s]
Epoc

In [33]:
models_eval_dict = {}
Ks = [5, 10]
i = 0

for model_class in models:
    for activation in activations:
        for lr in lrs:
            for latent in latent_dims:
                for λ in lambdas:
                    try:
                        model = load_model(model_class=model_class,
                                           activation=activation,
                                           hidden_size=latent,
                                           lr=lr,
                                           λ=λ,
                                           is_default_dataset=is_default_dataset)
                        for K in Ks:
                            (hits, ndcgs, mrrs) = evaluate_model(model, test_loader=val_loader, K=K)
                            models_eval_dict[f"row_{i}"] = [type(model).__name__, activation, latent, λ, lr, K, "HR", np.mean(hits)]
                            models_eval_dict[f"row_{i+1}"] = [type(model).__name__, activation, latent, λ, lr, K, "NDCG",np.mean(ndcgs)]
                            models_eval_dict[f"row_{i+2}"] = [type(model).__name__, activation, latent, λ, lr, K, "MRR",np.mean(mrrs)]
                            i += 3
                    except Exception as e:
                        print(e)
    #                     break
    #                 break
    #             break
    #         break
    #     break
    # break

AutoRecBase(
  (encoder): Linear(in_features=2876, out_features=10, bias=True)
  (act_1): PReLU(num_parameters=1)
  (decoder): Linear(in_features=10, out_features=2876, bias=True)
  (act_2): PReLU(num_parameters=1)
  (loss_func): MSELoss()
)
Epoch 0:   0%|          | 0/82 [14:31<?, ?it/s]
AutoRecBase(
  (encoder): Linear(in_features=2876, out_features=10, bias=True)
  (act_1): PReLU(num_parameters=1)
  (decoder): Linear(in_features=10, out_features=2876, bias=True)
  (act_2): PReLU(num_parameters=1)
  (loss_func): MSELoss()
)
AutoRecBase(
  (encoder): Linear(in_features=2876, out_features=10, bias=True)
  (act_1): PReLU(num_parameters=1)
  (decoder): Linear(in_features=10, out_features=2876, bias=True)
  (act_2): PReLU(num_parameters=1)
  (loss_func): MSELoss()
)
AutoRecBase(
  (encoder): Linear(in_features=2876, out_features=10, bias=True)
  (act_1): PReLU(num_parameters=1)
  (decoder): Linear(in_features=10, out_features=2876, bias=True)
  (act_2): PReLU(num_parameters=1)
  (loss_fun

In [42]:
columns = ["model", "activation", "latent_dim", "lambda", "lr","topk","metric","score"]
eval_df = pd.DataFrame.from_dict(models_eval_dict, orient='index', columns=columns)
eval_df.to_csv("obj/eval_df.csv", sep='\t')

eval_df

,model,activation,latent_dim,lambda,lr,topk,metric,score
row_0,AutoRecBase,<class 'torch.nn.modules.activation.PReLU'>,10,0.01,0.001,5,HR,0.0
row_1,AutoRecBase,<class 'torch.nn.modules.activation.PReLU'>,10,0.01,0.001,5,NDCG,0.0
row_2,AutoRecBase,<class 'torch.nn.modules.activation.PReLU'>,10,0.01,0.001,5,MRR,0.0
row_3,AutoRecBase,<class 'torch.nn.modules.activation.PReLU'>,10,0.01,0.001,10,HR,0.0
row_4,AutoRecBase,<class 'torch.nn.modules.activation.PReLU'>,10,0.01,0.001,10,NDCG,0.0
...,...,...,...,...,...,...,...,...
row_211,AutoRecBase,<class 'torch.nn.modules.activation.PReLU'>,300,100.00,0.010,5,NDCG,0.0
row_212,AutoRecBase,<class 'torch.nn.modules.activation.PReLU'>,300,100.00,0.010,5,MRR,0.0
row_213,AutoRecBase,<class 'torch.nn.modules.activation.PReLU'>,300,100.00,0.010,10,HR,0.0
row_214,AutoRecBase,<class 'torch.nn.modules.activation.PReLU'>,300,100.00,0.010,10,NDCG,0.0


In [43]:
eval_df.to_csv("obj/eval_df.csv", sep='\t')